### Package Installation

In [27]:
!pip install -r requirements.txt
!pip install -q optuna


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Imports

In [28]:
import numpy as np
import pandas as pd
import warnings
import random
import os
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import average_precision_score
import lightgbm as lgb
import optuna

from lightgbm import early_stopping, log_evaluation

In [29]:
warnings.filterwarnings('ignore')
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

## Loading Train and Test Data

In [61]:
URL_TRAIN = "https://www.mxhackathon.co.za/docs/TrainData.csv"
URL_TEST  = "https://www.mxhackathon.co.za/docs/TestData.csv"
dfTrain = pd.read_csv(URL_TRAIN)
dfTest  = pd.read_csv(URL_TEST)
print("Data successfully loaded:", dfTrain.shape, dfTest.shape)

Data successfully loaded: (73286, 21) (15611, 17)


##  Targeting and dropping Leakage Columns

In [31]:
leak_cols = ["InFinanceProcessSystemApp", "FinanceApplied", "FinanceApproved"]
dfTrain.drop(columns=leak_cols, inplace=True, errors="ignore")

## Feature Engineering

In [32]:
def feature_engineering(df):
    df = df.copy()
    df["DTLeadCreated"] = pd.to_datetime(df["DTLeadCreated"], errors='coerce')
    df["DTLeadAllocated"] = pd.to_datetime(df["DTLeadAllocated"], errors='coerce')
    
    # Enhanced temporal features
    df["LeadCreationHour"] = df["DTLeadCreated"].dt.hour
    df["LeadCreationDay"] = df["DTLeadCreated"].dt.dayofweek
    df["LeadCreationMonth"] = df["DTLeadCreated"].dt.month
    df["LeadCreationDayOfMonth"] = df["DTLeadCreated"].dt.day
    df["IsWeekend"] = df["LeadCreationDay"].isin([5, 6]).astype(int)
    df["IsMorning"] = (df["LeadCreationHour"] >= 8) & (df["LeadCreationHour"] <= 12)
    df["IsAfternoon"] = (df["LeadCreationHour"] > 12) & (df["LeadCreationHour"] <= 17)
    df["IsEvening"] = (df["LeadCreationHour"] > 17) | (df["LeadCreationHour"] < 8)
    
    # Time difference features
    df["AllocationLagHours"] = (df["DTLeadAllocated"] - df["DTLeadCreated"]).dt.total_seconds() / 3600
    df["AllocationLagMinutes"] = (df["DTLeadAllocated"] - df["DTLeadCreated"]).dt.total_seconds() / 60
    
    # Email domain features
    df["EmailDomainLength"] = df["Domain"].astype(str).apply(len)
    df["IsGmailOrYahoo"] = df["Domain"].str.lower().isin(["gmail.com", "yahoo.com"]).astype(int)
    
    # Dropping datetime and text-heavy columns
    drop_cols = ["DTLeadCreated", "DTLeadAllocated", "OBSFullName", "OBSEmail", "CustomerID"]
    df.drop(columns=drop_cols, inplace=True, errors="ignore")
    return df

### Applying feature_engineering to datasets

In [33]:
dfTrain = feature_engineering(dfTrain)
dfTest = feature_engineering(dfTest)

## Adding frequency encoding for important categorical features

In [34]:
for col in ["LeadCreationHour", "LeadCreationDay", "Domain", "LeadSource", "Make", "Model"]:
    if col in dfTrain.columns:
        freq = dfTrain[col].value_counts() / len(dfTrain)
        dfTrain[col + "_freq"] = dfTrain[col].map(freq)
        # Using training distributions for test data to avoid data leakage
        dfTest[col + "_freq"] = dfTest[col].map(freq).fillna(0)

## Creating interaction features

In [35]:
dfTrain["Lag_per_EmailLen"] = dfTrain["AllocationLagHours"] / (dfTrain["EmailDomainLength"] + 1)
dfTest["Lag_per_EmailLen"] = dfTest["AllocationLagHours"] / (dfTest["EmailDomainLength"] + 1)

if "Make" in dfTrain.columns and "Model" in dfTrain.columns:
    dfTrain["Make_Model_Interaction"] = dfTrain["Make"].astype(str) + "_" + dfTrain["Model"].astype(str)
    dfTest["Make_Model_Interaction"] = dfTest["Make"].astype(str) + "_" + dfTest["Model"].astype(str)

In [36]:
all_data = pd.concat([dfTrain.drop(columns="VehicleSold"), dfTest], axis=0)
cat_cols = all_data.select_dtypes(include="object").columns

## Encoding categorical features

In [37]:
encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    all_data[col] = le.fit_transform(all_data[col].astype(str))
    encoders[col] = le

X_train = all_data.iloc[:len(dfTrain), :]
X_test = all_data.iloc[len(dfTrain):, :]
y_train = dfTrain["VehicleSold"]

### Train/Test split

In [38]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=seed)

In [39]:
def objective(trial):
    params = {
        "objective": "binary",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "random_state": seed,
        "n_estimators": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "num_leaves": trial.suggest_int("num_leaves", 16, 256),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 100),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 10),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 5),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 5)
    }

    model = lgb.LGBMClassifier(**params)

    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val, y_val)],
        eval_metric="average_precision",
        callbacks=[
            early_stopping(stopping_rounds=50),
            log_evaluation(period=0)
        ]
    )

    preds = model.predict_proba(X_val)[:, 1]
    score = average_precision_score(y_val, preds)
    return score


## Running Optimization

In [40]:
# this will take about 5 min
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=45)

[I 2025-07-14 17:36:38,718] A new study created in memory with name: no-name-52a69656-9d61-4752-ae94-0a90ec13c096


Training until validation scores don't improve for 50 rounds


[I 2025-07-14 17:36:39,034] Trial 0 finished with value: 0.07886955063893974 and parameters: {'learning_rate': 0.09057914218578224, 'num_leaves': 46, 'min_child_samples': 19, 'feature_fraction': 0.8042082363356532, 'bagging_fraction': 0.8012015072878704, 'bagging_freq': 3, 'reg_alpha': 3.176542763261815, 'reg_lambda': 2.5295023877215788}. Best is trial 0 with value: 0.07886955063893974.


Early stopping, best iteration is:
[18]	valid_0's average_precision: 0.0788696	valid_0's binary_logloss: 0.174586
Training until validation scores don't improve for 50 rounds


[I 2025-07-14 17:36:39,590] Trial 1 finished with value: 0.08028379602642843 and parameters: {'learning_rate': 0.08140109882828014, 'num_leaves': 177, 'min_child_samples': 98, 'feature_fraction': 0.773822291842254, 'bagging_fraction': 0.8815219843799699, 'bagging_freq': 1, 'reg_alpha': 4.831712819343149, 'reg_lambda': 1.1513995366184133}. Best is trial 1 with value: 0.08028379602642843.


...

Early stopping, best iteration is:
[35]	valid_0's average_precision: 0.0789663	valid_0's binary_logloss: 0.174347
Training until validation scores don't improve for 50 rounds


[I 2025-07-14 17:37:20,306] Trial 43 finished with value: 0.08385273902822113 and parameters: {'learning_rate': 0.03799187763199631, 'num_leaves': 223, 'min_child_samples': 88, 'feature_fraction': 0.6018539385131686, 'bagging_fraction': 0.9239752040448184, 'bagging_freq': 4, 'reg_alpha': 4.963226993715288, 'reg_lambda': 4.20910134720847}. Best is trial 8 with value: 0.08441214916196588.


Early stopping, best iteration is:
[70]	valid_0's average_precision: 0.0838527	valid_0's binary_logloss: 0.173795
Training until validation scores don't improve for 50 rounds


[I 2025-07-14 17:37:21,543] Trial 44 finished with value: 0.08380879505799238 and parameters: {'learning_rate': 0.030248775622088206, 'num_leaves': 255, 'min_child_samples': 88, 'feature_fraction': 0.6752727266271814, 'bagging_fraction': 0.868272259292451, 'bagging_freq': 4, 'reg_alpha': 3.402778526945394, 'reg_lambda': 4.164819112158622}. Best is trial 8 with value: 0.08441214916196588.


Early stopping, best iteration is:
[32]	valid_0's average_precision: 0.0838088	valid_0's binary_logloss: 0.175239


### Cross-Fold Ensembling with Best Parameters

In [41]:
best_params = study.best_params
best_params.update({"objective": "binary", "metric": "average_precision", "verbosity": -1, "seed": seed})

## Calculating class weight for imbalance handling

In [42]:
from collections import Counter
counter = Counter(y_train)
scale_pos_weight = counter[0] / counter[1]
print("Scale pos weight:", scale_pos_weight)

# Adding scale_pos_weight to best params
best_params["scale_pos_weight"] = scale_pos_weight

Scale pos weight: 21.703221809169765


### Training with cross-validation and create model ensemble

In [43]:
n_folds = 5
kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=seed)
cv_models = []
cv_scores = []
oof_preds = np.zeros(len(X_train))

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
    X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    fold_model = lgb.LGBMClassifier(**best_params)
    fold_model.fit(
        X_fold_train, y_fold_train,
        eval_set=[(X_fold_val, y_fold_val)],
        eval_metric="average_precision",
        callbacks=[
            early_stopping(stopping_rounds=100),
            log_evaluation(period=0)
        ]
    )
    
    # Store out-of-fold predictions for threshold optimization
    oof_preds[val_idx] = fold_model.predict_proba(X_fold_val)[:, 1]
    
    fold_preds = fold_model.predict_proba(X_fold_val)[:, 1]
    fold_score = average_precision_score(y_fold_val, fold_preds)
    print(f"Fold {fold+1} PR-AUC: {fold_score:.6f}")
    
    cv_models.append(fold_model)
    cv_scores.append(fold_score)

print(f"Mean CV PR-AUC: {np.mean(cv_scores):.6f} ± {np.std(cv_scores):.6f}")

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[10]	valid_0's average_precision: 0.0792678
Fold 1 PR-AUC: 0.079268
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[42]	valid_0's average_precision: 0.0749151
Fold 2 PR-AUC: 0.074915
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[22]	valid_0's average_precision: 0.0729026
Fold 3 PR-AUC: 0.072903
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[24]	valid_0's average_precision: 0.0790928
Fold 4 PR-AUC: 0.079093
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[10]	valid_0's average_precision: 0.073329
Fold 5 PR-AUC: 0.073329
Mean CV PR-AUC: 0.075901 ± 0.002760


### Training single model on all data

In [44]:
final_model = lgb.LGBMClassifier(**best_params)
final_model.fit(X_train, y_train)

LGBMClassifier(bagging_fraction=0.8827946786278817, bagging_freq=2,
               feature_fraction=0.6514564127326442,
               learning_rate=0.061503873955098765, metric='average_precision',
               min_child_samples=99, num_leaves=100, objective='binary',
               reg_alpha=2.195634618382334, reg_lambda=1.845241969413674,
               scale_pos_weight=21.703221809169765, seed=42, verbosity=-1)

## Generating ensemble predictions (average of CV models)

In [45]:
cv_test_preds = np.zeros(len(X_test))
for model in cv_models:
    cv_test_preds += model.predict_proba(X_test)[:, 1]
cv_test_preds /= len(cv_models)

## Generating single model predictions

In [46]:
single_test_preds = final_model.predict_proba(X_test)[:, 1]

### Trying different ensemble weights to find optimal blend

In [47]:
blend_weights = [0.5, 0.6, 0.7, 0.8, 0.9]
blend_preds_dict = {}
for weight in blend_weights:
    blend_preds_dict[weight] = weight * cv_test_preds + (1-weight) * single_test_preds

### Defaulting to 0.8 weight for ensemble

In [48]:
probabilities = 0.8 * cv_test_preds + 0.2 * single_test_preds

## Feature importance analysis

In [49]:
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': final_model.feature_importances_
}).sort_values(by='Importance', ascending=False)
print("Top 20 features by importance:")
print(feature_importance.head(20))

Top 20 features by importance:
                   Feature  Importance
0                   LeadID         993
6            InterestModel         896
1                   Dealer         756
2               LeadSource         653
8               CellPrefix         623
27         LeadSource_freq         620
10           HourOfEnquiry         615
11            DayOfEnquiry         610
5             InterestMake         585
28        Lag_per_EmailLen         553
20      AllocationLagHours         550
24   LeadCreationHour_freq         499
13         LeadCreationDay         332
15  LeadCreationDayOfMonth         257
7                   Domain         216
26             Domain_freq         197
25    LeadCreationDay_freq         191
3                 LeadType         186
21    AllocationLagMinutes         163
12        LeadCreationHour         150


## Predicting On Test Data

In [50]:
dfTest["LeadID"] = dfTest["LeadID"].astype(int)

In [51]:
def save_submission(df: pd.DataFrame, probs: pd.Series | np.ndarray):
    """
    Writes the CSV exactly as the leaderboard expects.
    """
    out = pd.DataFrame(
        {
            "LeadID":                 df["LeadID"].values,
            "VehicleSoldProbability": probs
        }
    )

    # minimal sanity checks
    assert len(out) == len(df),                "Length mismatch."
    assert out["VehicleSoldProbability"].between(0, 1).all(), "Probs out of range."
    assert out["LeadID"].is_unique,            "Duplicate LeadID values."
    assert not out.isna().any().any(),         "NaNs detected."

    fname = f"submission.csv"
    out.to_csv(fname, index=False)

save_submission(dfTest, probabilities)